##### Dependencies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

##### Get the data

Para trabalhar com os dados foi necessário fazer o mapeamento dos valores de texto de 'Target' para inteiros, ficando com 'Dropout' = 0, 'Graduate' = 1 e 'Enrolled' = 2. Como os valores dos atributos são todos numéricos, não foi necessário fazer nenhum tratamento sobre os mesmos. 

In [ ]:
raw_data = pd.read_csv('Dataset_Trabalho.csv', sep=';')


print(raw_data.info())

##### Plot with the distribution of samples for each label (class)

Neste passo, fizemos um gráfico no qual é possível observar a distribuição do número de amostras para cada valor de 'Target'. Com isto conseguimos verificar que o nosso dataset tem um problema de balanceamento de classes (class imbalance), ou seja, o número de amostras não é igual para todas. 

In [ ]:
sns.set_theme(style='dark')

ax = sns.countplot(x='Target', data=raw_data)
plt.title(f'Classes distribution')
plt.xlabel('Classes')
plt.show()

# Extract the counts from the countplot
x_labels = [tick.get_text() for tick in ax.get_xticklabels()]
counts = [rect.get_height() for rect in ax.patches]
print("Distribution [Dropout, Graduate, Enrolled]: ", counts)

##### Data Normalisation

Como os atributos do dataset têm valores muito variados, aplicámos normalização dos dados, para melhorar a performance do modelo e para permitir uma melhor comparação entre os atributos.

In [ ]:
# Convert all the values to numeric
# Dictionary to map string values to integers
mapping = {'Dropout': 0, 'Graduate': 1, 'Enrolled': 2}

# Substitute string values with integers
raw_data['Target'] = raw_data['Target'].map(mapping)

# copy the data
normalized_data = raw_data.copy()

# apply normalization techniques
for column in normalized_data.columns:
	normalized_data[column] = (normalized_data[column] - normalized_data[column].min()) / (normalized_data[column].max() - normalized_data[column].min())	

# view normalized data
normalized_data



##### Correlation Matrix

Analisámos também a matrix de correlação dos atributos, para conseguir visualizar melhor a relação de cada um com o 'Target'. Valores mais altos entre 2 atributos indicam uma relação diretamente proporcional ente eles, enquanto valores mais baixos indicam proporcionalidade inversa.

In [ ]:
corr = raw_data.corr()
sns.set_theme(style='dark')
f, ax = plt.subplots(figsize=(30, 30))
sns.heatmap(corr, annot=True, ax=ax)
plt.title('Correlation Matrix', fontsize=14)
plt.show()

##### Subsets definition

De seguida, separámos as variáveis (atributos) independentes (X) dos dependentes (y). 

Como o dataset tinha um número muito elevado de atributos (36) decidimos utilizar apenas os mais relevantes para o cálculo do objetivo. Após algumas tentativas e comparações, escolhemos usar 18 destes atributos.

Depois dividimos X e y em sets de treino e teste, cada um com 80% e 20% do total de amostras, respetivamente.

In [ ]:
columns_names = raw_data.values[0, :-1]
X = np.array(raw_data.values[:, :-1])
X_with_col = pd.DataFrame(X, columns=columns_names)
y = raw_data.values[:, -1]

selector = SelectKBest(f_classif, k=18)
X_new = selector.fit_transform(X_with_col, y)

train_feature_indices = selector.get_support(indices=True)
print(f"Indices of the selected features: {train_feature_indices}")

# Normalisation
scaler = StandardScaler()  	
scaler.fit(X_new) 

# splitting X and y into training and testing sets
X_train, X_test,\
	y_train, y_test = train_test_split(X_new, y,
									test_size=0.2,
									random_state=1)
ax = sns.countplot(data=y_train)
plt.title('Total number of samples in y_train')
plt.show()

##### Class imbalance treatment

Por fim, para resolver o problema das amostras desequilibradas para cada classe, utilizámos SMOTE (Synthetic Minority Oversampling Technique), para aumentar o número de amostras das classes em minoria.

Optámos por esta técnica, porque ao invés de apenas duplicar amostras já existentes até alcançar a quantidade desejada, gera novas amostras sintéticas encontrando os k-nearest neighbors mais próximos de cada amostra das classes em minoria e selecionando aleatoriamente um ou mais destes.

In [ ]:
smot = SMOTE()
X_train, y_train = smot.fit_resample(X_train, y_train)
# # equal sampling now (check)
ax = sns.countplot(data=y_train)
plt.title(f'Total number of samples in y_train after SMOTE application')
plt.show()


##### Logistic Regression training and testing

In [ ]:
logreg = LogisticRegression(C=100, class_weight='balanced', max_iter=10000)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test) 

cnf_matrix = metrics.confusion_matrix(y_test, y_pred) 

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_matrix, annot=True, fmt="d", cmap="Reds")
plt.title("Confusion Matrix")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

print("Accuracy:",metrics.accuracy_score(y_test, y_pred)) 

print("Precision:",metrics.precision_score(y_test, y_pred, average='weighted', zero_division=1)) 

print("Recall:",metrics.recall_score(y_test, y_pred, average='weighted', zero_division=1))

##### Function to test new data

In [ ]:
def get_input():
    input_text = input_entry.get()
    return input_text

In [ ]:
def test_input():
    input_data = get_input()
    print(f"\nInput: {input_data}")
    columns_names = raw_data.values[0, :-1]
    
    X_values = np.array([input_data.split(";")], dtype=float)
    X_df = pd.DataFrame(X_values, columns=columns_names)

    X_selected = X_df.iloc[:, train_feature_indices]

    X_normalized = scaler.transform([X_selected.values[0, :]])

    result = logreg.predict(X_normalized) 
    print(f"\nResult: {result}")
    result_text = "" 
    if result[0] == 0.0:
        result_text = 'Dropout'
    elif result[0] == 1.0:
        result_text = 'Graduate'
    elif result[0] == 2.0:
        result_text = 'Enrolled'

    message_label.config(text=f"Target prediction: {result_text}")


    

### Interface

In [ ]:
from tkinter import *

##### Dados para teste

Marital status;Application mode;Application order;Course;"Daytime/evening attendance	";Previous qualification;Previous qualification (grade);Nacionality;Mother's qualification;Father's qualification;Mother's occupation;Father's occupation;Admission grade;Displaced;Educational special needs;Debtor;Tuition fees up to date;Gender;Scholarship holder;Age at enrollment;International;Curricular units 1st sem (credited);Curricular units 1st sem (enrolled);Curricular units 1st sem (evaluations);Curricular units 1st sem (approved);Curricular units 1st sem (grade);Curricular units 1st sem (without evaluations);Curricular units 2nd sem (credited);Curricular units 2nd sem (enrolled);Curricular units 2nd sem (evaluations);Curricular units 2nd sem (approved);Curricular units 2nd sem (grade);Curricular units 2nd sem (without evaluations);Unemployment rate;Inflation rate;GDP;Target

1;17;5;171;1;1;122.0;1;19;12;5;9;127.3;1;0;0;1;1;0;20;0;0;0;0;0;0.0;0;0;0;0;0;0.0;0;10.8;1.4;1.74;Dropout

In [ ]:
window = Tk()
window.title('SAD')
window.geometry("500x300")

In [ ]:
input_entry = Entry(window)
input_entry.pack(anchor='center', pady=10)

button = Button(window, text="Testar dados", command=test_input)
button.pack(anchor='center', pady=10)

message_label = Label(window, text="")
message_label.pack(anchor='center', pady=10)

##### Main loop

In [ ]:
window.mainloop()